In [15]:
# Part 1: Setup and Import Libraries
import os
import requests
import time
from pathlib import Path
from urllib.parse import urljoin
import logging

# Setup logging to track download progress
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Create download directory
download_dir = "Bihar_Al_Anwaar_PDFs"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
    logger.info(f"Created directory: {download_dir}")
else:
    logger.info(f"Directory already exists: {download_dir}")

print("✅ Part 1 Complete: Libraries imported and download directory created")
print(f"📁 Download directory: {os.path.abspath(download_dir)}")

2025-08-17 21:45:47,322 - INFO - Directory already exists: Bihar_Al_Anwaar_PDFs


✅ Part 1 Complete: Libraries imported and download directory created
📁 Download directory: c:\Users\sadiq\OneDrive\Documents\projects\rag-for-ba\Bihar_Al_Anwaar_PDFs


In [16]:
# Part 2: Download Function
def download_pdf(url, filename, download_dir, max_retries=3):
    """
    Download a PDF file with retry mechanism
    """
    filepath = os.path.join(download_dir, filename)
    
    # Check if file already exists
    if os.path.exists(filepath):
        logger.info(f"⏭️  {filename} already exists, skipping...")
        return True
    
    for attempt in range(max_retries):
        try:
            logger.info(f"⬇️  Downloading {filename} (Attempt {attempt + 1}/{max_retries})")
            
            # Send GET request with headers to mimic browser
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'application/pdf,application/octet-stream,*/*',
                'Accept-Language': 'en-US,en;q=0.9',
                'Referer': 'https://hubeali.com/'
            }
            
            response = requests.get(url, headers=headers, stream=True, timeout=30)
            response.raise_for_status()
            
            # Check if response is actually a PDF
            content_type = response.headers.get('content-type', '').lower()
            if 'pdf' not in content_type and 'octet-stream' not in content_type:
                logger.warning(f"⚠️  Unexpected content type for {filename}: {content_type}")
            
            # Download the file
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            # Verify file size
            file_size = os.path.getsize(filepath)
            if file_size == 0:
                os.remove(filepath)
                raise Exception("Downloaded file is empty")
            
            logger.info(f"✅ {filename} downloaded successfully ({file_size:,} bytes)")
            return True
            
        except Exception as e:
            logger.error(f"❌ Failed to download {filename}: {str(e)}")
            if os.path.exists(filepath):
                os.remove(filepath)
            
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 2  # Progressive wait time
                logger.info(f"⏳ Waiting {wait_time} seconds before retry...")
                time.sleep(wait_time)
            else:
                logger.error(f"🚫 Failed to download {filename} after {max_retries} attempts")
                return False

def check_pdf_exists(url, timeout=10):
    """
    Check if a PDF URL exists without downloading
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.head(url, headers=headers, timeout=timeout)
        return response.status_code == 200
    except:
        return False

print("✅ Part 2 Complete: Download functions defined")

✅ Part 2 Complete: Download functions defined


In [17]:
# Part 3: URL Generation and Validation
def generate_pdf_urls(start_volume=1, end_volume=94):
    """
    Generate PDF URLs for all volumes
    """
    base_url = "https://hubeali.com/books/English-Books/BiharAlAnwaar/BiharAlAnwaar_V{}.pdf"
    urls = []
    
    for volume in range(start_volume, end_volume + 1):
        url = base_url.format(volume)
        filename = f"BiharAlAnwaar_V{volume}.pdf"
        urls.append((url, filename, volume))
    
    return urls

# Generate all URLs
pdf_urls = generate_pdf_urls(1, 94)
logger.info(f"📋 Generated {len(pdf_urls)} PDF URLs")

# Display first few URLs for verification
print("🔍 First 5 URLs to download:")
for i, (url, filename, volume) in enumerate(pdf_urls[:5]):
    print(f"  {volume:2d}. {url}")

print(f"\n📊 Total volumes to download: {len(pdf_urls)}")
print(f"💾 Download directory: {os.path.abspath(download_dir)}")

2025-08-17 21:45:47,346 - INFO - 📋 Generated 94 PDF URLs


🔍 First 5 URLs to download:
   1. https://hubeali.com/books/English-Books/BiharAlAnwaar/BiharAlAnwaar_V1.pdf
   2. https://hubeali.com/books/English-Books/BiharAlAnwaar/BiharAlAnwaar_V2.pdf
   3. https://hubeali.com/books/English-Books/BiharAlAnwaar/BiharAlAnwaar_V3.pdf
   4. https://hubeali.com/books/English-Books/BiharAlAnwaar/BiharAlAnwaar_V4.pdf
   5. https://hubeali.com/books/English-Books/BiharAlAnwaar/BiharAlAnwaar_V5.pdf

📊 Total volumes to download: 94
💾 Download directory: c:\Users\sadiq\OneDrive\Documents\projects\rag-for-ba\Bihar_Al_Anwaar_PDFs


In [18]:
# Part 4: Quick URL Validation (Optional - you can skip this if you want to start downloading immediately)
def validate_sample_urls(sample_size=5):
    """
    Validate a sample of URLs to ensure they exist before starting bulk download
    """
    print(f"🔍 Validating {sample_size} sample URLs...")
    
    # Test first few, middle, and last few volumes
    sample_indices = []
    if len(pdf_urls) >= sample_size:
        sample_indices = [0, 1, len(pdf_urls)//2, len(pdf_urls)-2, len(pdf_urls)-1]
    else:
        sample_indices = list(range(len(pdf_urls)))
    
    valid_count = 0
    for i in sample_indices[:sample_size]:
        url, filename, volume = pdf_urls[i]
        print(f"  Testing Volume {volume}...", end=" ")
        if check_pdf_exists(url):
            print("✅ Available")
            valid_count += 1
        else:
            print("❌ Not found")
        time.sleep(1)  # Be nice to the server
    
    print(f"\n📊 Validation Results: {valid_count}/{sample_size} URLs are accessible")
    
    if valid_count == 0:
        print("⚠️  Warning: No URLs were accessible. Please check the base URL pattern.")
        return False
    elif valid_count < sample_size:
        print("⚠️  Warning: Some URLs may not be accessible. This is normal if some volumes don't exist.")
    else:
        print("✅ All sample URLs are accessible!")
    
    return True

# Run validation (comment this out if you want to skip validation)
validation_result = validate_sample_urls(5)

🔍 Validating 5 sample URLs...
  Testing Volume 1... ✅ Available
  Testing Volume 2... ✅ Available
  Testing Volume 48... ✅ Available
  Testing Volume 93... ✅ Available
  Testing Volume 94... ✅ Available

📊 Validation Results: 5/5 URLs are accessible
✅ All sample URLs are accessible!


In [19]:
# Part 5: Mass Download with Progress Tracking
def download_all_volumes(pdf_urls, download_dir, delay_between_downloads=2):
    """
    Download all PDF volumes with progress tracking
    """
    total_volumes = len(pdf_urls)
    successful_downloads = 0
    failed_downloads = []
    
    print(f"🚀 Starting download of {total_volumes} volumes...")
    print(f"📁 Saving to: {os.path.abspath(download_dir)}")
    print(f"⏱️  Delay between downloads: {delay_between_downloads} seconds")
    print("-" * 60)
    
    start_time = time.time()
    
    for i, (url, filename, volume) in enumerate(pdf_urls, 1):
        print(f"\n📖 Progress: {i}/{total_volumes} - Volume {volume}")
        
        success = download_pdf(url, filename, download_dir)
        
        if success:
            successful_downloads += 1
        else:
            failed_downloads.append((volume, filename, url))
        
        # Progress update
        progress_percentage = (i / total_volumes) * 100
        print(f"📊 Overall Progress: {progress_percentage:.1f}% ({successful_downloads} successful, {len(failed_downloads)} failed)")
        
        # Delay between downloads to be respectful to the server
        if i < total_volumes:  # Don't delay after the last download
            time.sleep(delay_between_downloads)
    
    # Final summary
    elapsed_time = time.time() - start_time
    print("\n" + "="*60)
    print("📋 DOWNLOAD SUMMARY")
    print("="*60)
    print(f"✅ Successful downloads: {successful_downloads}/{total_volumes}")
    print(f"❌ Failed downloads: {len(failed_downloads)}")
    print(f"⏱️  Total time: {elapsed_time/60:.1f} minutes")
    print(f"📁 Files saved to: {os.path.abspath(download_dir)}")
    
    if failed_downloads:
        print(f"\n❌ Failed Downloads:")
        for volume, filename, url in failed_downloads:
            print(f"  - Volume {volume}: {filename}")
    
    return successful_downloads, failed_downloads

# Start the download process
print("🎯 Ready to start downloading!")
print("⚠️  This will download up to 94 PDF files. Make sure you have sufficient disk space.")
print("💡 You can interrupt the process with Ctrl+C if needed.")

# Uncomment the line below when you're ready to start downloading
successful, failed = download_all_volumes(pdf_urls, download_dir, delay_between_downloads=2)

2025-08-17 21:45:54,242 - INFO - ⬇️  Downloading BiharAlAnwaar_V1.pdf (Attempt 1/3)


🎯 Ready to start downloading!
⚠️  This will download up to 94 PDF files. Make sure you have sufficient disk space.
💡 You can interrupt the process with Ctrl+C if needed.
🚀 Starting download of 94 volumes...
📁 Saving to: c:\Users\sadiq\OneDrive\Documents\projects\rag-for-ba\Bihar_Al_Anwaar_PDFs
⏱️  Delay between downloads: 2 seconds
------------------------------------------------------------

📖 Progress: 1/94 - Volume 1


2025-08-17 21:45:55,143 - INFO - ✅ BiharAlAnwaar_V1.pdf downloaded successfully (2,282,181 bytes)


📊 Overall Progress: 1.1% (1 successful, 0 failed)


2025-08-17 21:45:57,144 - INFO - ⬇️  Downloading BiharAlAnwaar_V2.pdf (Attempt 1/3)


...

📊 Overall Progress: 98.9% (93 successful, 0 failed)


2025-08-17 21:53:50,948 - INFO - ⬇️  Downloading BiharAlAnwaar_V94.pdf (Attempt 1/3)



📖 Progress: 94/94 - Volume 94


2025-08-17 21:53:53,059 - INFO - ✅ BiharAlAnwaar_V94.pdf downloaded successfully (6,726,593 bytes)


📊 Overall Progress: 100.0% (94 successful, 0 failed)

📋 DOWNLOAD SUMMARY
✅ Successful downloads: 94/94
❌ Failed downloads: 0
⏱️  Total time: 8.0 minutes
📁 Files saved to: c:\Users\sadiq\OneDrive\Documents\projects\rag-for-ba\Bihar_Al_Anwaar_PDFs


In [21]:
# Part 6: Retry Failed Downloads and Utilities
def retry_failed_downloads(failed_downloads, download_dir, max_retries=3):
    """
    Retry downloading failed volumes
    """
    if not failed_downloads:
        print("✅ No failed downloads to retry!")
        return [], []
    
    print(f"🔄 Retrying {len(failed_downloads)} failed downloads...")
    
    successful_retries = 0
    still_failed = []
    
    for volume, filename, url in failed_downloads:
        print(f"\n🔄 Retrying Volume {volume}...")
        success = download_pdf(url, filename, download_dir, max_retries)
        
        if success:
            successful_retries += 1
            print(f"✅ Volume {volume} downloaded successfully on retry!")
        else:
            still_failed.append((volume, filename, url))
            print(f"❌ Volume {volume} still failed after retry")
        
        time.sleep(2)  # Delay between retries
    
    print(f"\n📊 Retry Summary:")
    print(f"✅ Successful retries: {successful_retries}")
    print(f"❌ Still failed: {len(still_failed)}")
    
    return successful_retries, still_failed

def check_downloaded_files(download_dir):
    """
    Check what files have been downloaded and their sizes
    """
    if not os.path.exists(download_dir):
        print(f"❌ Directory {download_dir} does not exist")
        return
    
    files = [f for f in os.listdir(download_dir) if f.endswith('.pdf')]
    files.sort(key=lambda x: int(x.split('_V')[1].split('.')[0]))  # Sort by volume number
    
    print(f"📁 Files in {download_dir}:")
    print("-" * 60)
    
    total_size = 0
    for file in files:
        filepath = os.path.join(download_dir, file)
        size = os.path.getsize(filepath)
        total_size += size
        volume_num = file.split('_V')[1].split('.')[0]
        print(f"📖 Volume {volume_num:>2}: {file} ({size:,} bytes)")
    
    print("-" * 60)
    print(f"📊 Total files: {len(files)}")
    print(f"💾 Total size: {total_size:,} bytes ({total_size/1024/1024:.1f} MB)")
    
    # Check for missing volumes
    expected_volumes = set(range(1, 95))
    downloaded_volumes = set()
    
    for file in files:
        try:
            volume_num = int(file.split('_V')[1].split('.')[0])
            downloaded_volumes.add(volume_num)
        except:
            pass
    
    missing_volumes = expected_volumes - downloaded_volumes
    if missing_volumes:
        missing_sorted = sorted(list(missing_volumes))
        print(f"\n❌ Missing volumes: {missing_sorted}")
    else:
        print("\n✅ All volumes (1-94) have been downloaded!")

# Utility functions
def get_download_status():
    """Quick status check"""
    check_downloaded_files(download_dir)

print("✅ Part 6 Complete: Retry functions and utilities defined")
print("\n🎯 To start downloading, uncomment this line in Part 5:")
print("# successful, failed = download_all_volumes(pdf_urls, download_dir, delay_between_downloads=2)")
print("\n📋 Available utility functions:")
print("- get_download_status() - Check what's been downloaded")
print("- retry_failed_downloads(failed_list, download_dir) - Retry failed downloads")
print("- check_downloaded_files(download_dir) - Detailed file analysis")

✅ Part 6 Complete: Retry functions and utilities defined

🎯 To start downloading, uncomment this line in Part 5:
# successful, failed = download_all_volumes(pdf_urls, download_dir, delay_between_downloads=2)

📋 Available utility functions:
- get_download_status() - Check what's been downloaded
- retry_failed_downloads(failed_list, download_dir) - Retry failed downloads
- check_downloaded_files(download_dir) - Detailed file analysis


In [22]:
get_download_status()

📁 Files in Bihar_Al_Anwaar_PDFs:
------------------------------------------------------------
📖 Volume  1: BiharAlAnwaar_V1.pdf (2,282,181 bytes)
📖 Volume  2: BiharAlAnwaar_V2.pdf (4,203,450 bytes)
📖 Volume  3: BiharAlAnwaar_V3.pdf (3,710,001 bytes)
📖 Volume  4: BiharAlAnwaar_V4.pdf (2,951,998 bytes)
📖 Volume  5: BiharAlAnwaar_V5.pdf (3,397,830 bytes)
📖 Volume  6: BiharAlAnwaar_V6.pdf (3,641,659 bytes)
📖 Volume  7: BiharAlAnwaar_V7.pdf (3,881,179 bytes)
📖 Volume  8: BiharAlAnwaar_V8.pdf (4,046,060 bytes)
📖 Volume  9: BiharAlAnwaar_V9.pdf (3,595,931 bytes)
📖 Volume 10: BiharAlAnwaar_V10.pdf (4,955,702 bytes)
📖 Volume 11: BiharAlAnwaar_V11.pdf (4,062,586 bytes)
📖 Volume 12: BiharAlAnwaar_V12.pdf (2,891,832 bytes)
📖 Volume 13: BiharAlAnwaar_V13.pdf (3,170,662 bytes)
📖 Volume 14: BiharAlAnwaar_V14.pdf (3,767,734 bytes)
📖 Volume 15: BiharAlAnwaar_V15.pdf (2,861,611 bytes)
📖 Volume 16: BiharAlAnwaar_V16.pdf (2,888,307 bytes)
📖 Volume 17: BiharAlAnwaar_V17.pdf (3,118,462 bytes)
📖 Volume 18: B